***Dataset - https://www.kaggle.com/datasets/ikarus777/best-artworks-of-all-time****

**Import Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Define the Content and Style Layers**

In [2]:
content_layers = ['block4_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']


**Define the Content and Style Layers**

In [ ]:
import os

def load_dataset(dataset_path):
    image_paths = [os.path.join(dataset_path, filename) for filename in os.listdir(dataset_path)]
    images = [load_image(path) for path in image_paths]
    return images

dataset_path = "path/to/dataset"
content_images = load_dataset(os.path.join(dataset_path, "content"))
style_images = load_dataset(os.path.join(dataset_path, "style"))


**Build the Model**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers

def get_model(input_shape=(224, 224, 3), num_content_layers=1, num_style_layers=5):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    content_outputs = [base_model.get_layer(f'block{idx}_conv2').output for idx in range(1, num_content_layers + 1)]
    style_outputs = [base_model.get_layer(f'block{idx}_conv1').output for idx in range(1, num_style_layers + 1)]

    model_outputs = content_outputs + style_outputs

    return Model(inputs=base_model.input, outputs=model_outputs)

# Modify the number of content and style layers based on your requirements
num_content_layers = 1
num_style_layers = 5

model = get_model(num_content_layers=num_content_layers, num_style_layers=num_style_layers)
model.trainable = False


**Define Loss Functions**

In [4]:
def content_loss(base_content, target):
    # Assuming both base_content and target are 3D tensors (height, width, channels)
    return tf.reduce_mean(tf.square(base_content - target))

def gram_matrix(x):
    # Assuming x is a 3D tensor (height, width, channels)
    channels = int(x.shape[-1])
    a = tf.reshape(x, (-1, channels))
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

def style_loss(base_style, gram_target):
    # Assuming base_style is a 3D tensor (height, width, channels)
    return tf.reduce_mean(tf.square(gram_matrix(base_style) - gram_target))


**Define the Total Variation Loss**

In [5]:
def total_variation_loss(img, tv_weight=1e-6):
    return tv_weight * tf.reduce_sum(tf.image.total_variation(img))


**Generate the Model Input**

In [ ]:
num_generated_images = len(content_images)

generated_images = [
    tf.Variable(content_image, dtype=tf.float32)
    for _ in range(num_generated_images)
]


**Define the Optimizer**

In [ ]:
optimizer = tf.optimizers.Adam(
    learning_rate=0.001,  # You may need to adjust this based on your dataset and model
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
)


**Training Loop**

In [ ]:
def compute_loss(model, generated_image, content_target, style_targets, tv_weight=1e-6):
    model_outputs = model(generated_image)
    content_outputs = model_outputs[:len(content_layers)]
    style_outputs = model_outputs[len(content_layers):]

    content_loss_value = 0
    style_loss_value = 0

    for target_content, generated_content in zip(content_target, content_outputs):
        content_loss_value += content_loss(generated_content[0], target_content)

    for target_style, generated_style in zip(style_targets, style_outputs):
        style_loss_value += style_loss(generated_style[0], target_style)

    tv_loss_value = total_variation_loss(generated_image, tv_weight=tv_weight)

    total_loss = content_loss_value + style_loss_value + tv_loss_value
    return total_loss


**Training Loop**

In [ ]:
@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, image, content_target, style_targets)
        loss += total_variation_weight * total_variation_loss(image)

    grads = tape.gradient(loss, image)
    optimizer.apply_gradients([(grads, image)])

    # Clip the image values to be within the valid range [0.0, 1.0]
    image.assign(tf.clip_by_value(image, 0.0, 1.0))

# Training loop
for epoch in range(num_epochs):
    for generated_image in generated_images:
        train_step(generated_image)

# Access the generated images as NumPy arrays
final_images = [generated_image.numpy().squeeze() for generated_image in generated_images]


**Style Adaptation**

In [ ]:
def interpolate_styles(style1, style2, alpha):
    # Interpolate style features between two styles
    interpolated_style = alpha * style1 + (1 - alpha) * style2
    return interpolated_style


style1 = model.predict(style_image1)
style2 = model.predict(style_image2)

alpha = 0.5  # Adjust alpha for the desired interpolation level
new_style = interpolate_styles(style1, style2, alpha)



**Evaluation**


In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np


def load_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)


content_image = load_image("content.jpg")
style_image = load_image("style.jpg")
style transfer function
def style_transfer(content_image, style_image, model, num_epochs=10):
    generated_image = tf.Variable(content_image, dtype=tf.float32)



# Measure the time taken for style transfer
start_time = time.time()


model = get_model()
content_target = model.predict(content_image)
style_targets = model.predict(style_image)

# Perform style transfer
generated_image = style_transfer(content_image, style_image, model, num_epochs=10)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Style transfer took {elapsed_time} seconds.")
